In [36]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import StratifiedKFold
import joblib

lgb_train = lgb.Dataset(X[FEATS], y)
lgb_test = lgb.Dataset(test[FEATS], y_test)

In [37]:
model = lgb.train(
    {'objective': 'binary'}, 
    lgb_train,
    valid_sets=[lgb_test],
    verbose_eval=100,
    num_boost_round=10000,
    early_stopping_rounds=500,
)

preds = model.predict(test[FEATS])
acc = accuracy_score(y_test, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_test, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1320806, number of negative: 699865
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031773 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3382
[LightGBM] [Info] Number of data points in the train set: 2020671, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.653647 -> initscore=0.635110
[LightGBM] [Info] Start training from score 0.635110
Training until validation scores don't improve for 500 rounds
[100]	valid_0's binary_logloss: 0.604177
[200]	valid_0's binary_logloss: 0.59452
[300]	valid_0's binary_logloss: 0.588991
[400]	valid_0's binary_logloss: 0.584086
[500]	valid_0's binary_logloss: 0.581453
[600]	valid_0's binary_logloss: 0.576971
[700]	valid_0's binary_logloss: 0.574343
[800]	valid_0's binary_logloss: 0.571841
[900]	valid_0's binary_logloss: 0.571095
[1000]

In [4]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import StratifiedKFold
import joblib
import pandas as pd
from tqdm import tqdm

In [13]:
test = pd.read_csv('../../../data/test.csv')
sub = test[test['answerCode'] == -1]

In [14]:
p = []

FEATS = [
       'KnowledgeTag', 'month', 'hour', 'week', 'elapsed', 'elapsed_cate',
       'assessmentItemID0', 'assessmentItemID1', 'assessmentItemID2',
        'tag_mean', 'tag_std', 'ass0_mean', 'ass0_std',
       'ass1_mean', 'ass1_std', 'ass2_mean', 'ass2_std',
       'user_total_answer']

for i in tqdm(range(5)):
    load_model = joblib.load(f'lgb_{i}.pkl')
    preds = load_model.predict(sub[FEATS])
    p.append(preds)

#preds = model.predict(sub[FEATS])

100%|██████████| 5/5 [00:00<00:00,  9.33it/s]


In [15]:
s = pd.read_csv('../output/submission.csv')
m = (p[0] + p[1] + p[2] + p[3] + p[4])/5
s['prediction'] = m

In [16]:
s.to_csv('../output/submission.csv', index=False)

In [7]:
from tqdm import tqdm

n_splits = 5
str_kf = StratifiedKFold(n_splits = n_splits)

y = df['answerCode']
X = df.drop(['answerCode'], axis=1)

for i, (train_index, test_index) in tqdm(enumerate(str_kf.split(X, y))):
    X_train, X_valid = X.loc[train_index], X.loc[test_index]
    y_train, y_valid = y.loc[train_index], y.loc[test_index]
    
    lgb_train = lgb.Dataset(X_train[FEATS], y_train)
    lgb_test = lgb.Dataset(X_valid[FEATS], y_valid)
    
    model = lgb.train(
        {'objective': 'binary'}, 
        lgb_train,
        valid_sets=[lgb_train, lgb_test],
        verbose_eval=1000,
        num_boost_round=20000,
        early_stopping_rounds=500,
    )

    preds = model.predict(X_valid[FEATS])
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)

    print(f'VALID AUC : {auc} ACC : {acc}\n')
    
    joblib.dump(model, f'lgb_{i}.pkl')

0it [00:00, ?it/s]/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1322870, number of negative: 697894
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026835 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2617
[LightGBM] [Info] Number of data points in the train set: 2020764, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654639 -> initscore=0.639492
[LightGBM] [Info] Start training from score 0.639492
Training until validation scores don't improve for 500 rounds
[1000]	training's binary_logloss: 0.502915	valid_1's binary_logloss: 0.522087
[2000]	training's binary_logloss: 0.488195	valid_1's binary_logloss: 0.516882
[3000]	training's binary_logloss: 0.477446	valid_1's binary_logloss: 0.515208
[4000]	training's binary_logloss: 0.468539	valid_1's binary_logloss: 0.514793
[5000]	training's binary_logloss: 0.460291	valid_1's binary_lo

1it [04:29, 269.48s/it]/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1322870, number of negative: 697895
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026458 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2617
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654638 -> initscore=0.639490
[LightGBM] [Info] Start training from score 0.639490
Training until validation scores don't improve for 500 rounds
[1000]	training's binary_logloss: 0.504509	valid_1's binary_logloss: 0.514961
[2000]	training's binary_logloss: 0.489331	valid_1's binary_logloss: 0.509794
[3000]	training's binary_logloss: 0.478414	valid_1's binary_logloss: 0.507531
[4000]	training's binary_logloss: 0.469049	valid_1's binary_logloss: 0.506921
Early stopping, best iteration is:
[3620]	training's binary_lo

2it [08:11, 255.14s/it]

...

3it [12:43, 260.20s/it]/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1322871, number of negative: 697894
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026056 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2617
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654639 -> initscore=0.639492
[LightGBM] [Info] Start training from score 0.639492
Training until validation scores don't improve for 500 rounds
[1000]	training's binary_logloss: 0.504961	valid_1's binary_logloss: 0.512287
[2000]	training's binary_logloss: 0.48983	valid_1's binary_logloss: 0.506127
[3000]	training's binary_logloss: 0.479019	valid_1's binary_logloss: 0.503517
[4000]	training's binary_logloss: 0.469705	valid_1's binary_logloss: 0.502388
[5000]	training's binary_logloss: 0.461417	valid_1's binary_log

4it [17:28, 267.74s/it]/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1322871, number of negative: 697894
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026189 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2617
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654639 -> initscore=0.639492
[LightGBM] [Info] Start training from score 0.639492
Training until validation scores don't improve for 500 rounds
[1000]	training's binary_logloss: 0.504479	valid_1's binary_logloss: 0.517238
[2000]	training's binary_logloss: 0.489148	valid_1's binary_logloss: 0.511267
[3000]	training's binary_logloss: 0.478538	valid_1's binary_logloss: 0.509733
[4000]	training's binary_logloss: 0.469674	valid_1's binary_logloss: 0.509303
Early stopping, best iteration is:
[3945]	training's binary_lo

5it [21:24, 256.85s/it]
